In [ ]:
pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=6006bd8d2001a48b27d9b67847f5340ba462fde3bdff93dfdf002d80f66b7745
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import glob
import re
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import FunctionTransformer
import emoji


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# change directory to the folder that contains csv files
%cd /content/drive/MyDrive/NetServ/Sample_Data/v2/

# Get the file names
filenames = glob.glob("*.csv") # assuming the files are in the current directory

# Sort the file names in alphabetical order
filenames.sort()

# Read and concatenate the files
dfs = [] # an empty list to store the data frames

# loop through each file name and read it as a data frame
for filename in filenames:
    df = pd.read_csv(filename) # read each file with header
    dfs.append(df) # append each data frame to the list

# Concatenate all data frames in the list
big_df = pd.concat(dfs, ignore_index=True)

# Display the merged data frame
print(big_df)

/content/drive/MyDrive/NetServ/Sample_Data/v2
                                          review_comment  rating
0      Lost in Hartsfield I swear I hate ATL airport ...       1
1                                Atlanta airport trash 😫       1
2      flight got fucking canceled, might have to spe...       1
3      Disgusting.  Took one bite, spit it out and th...       1
4      don’t ever get lost in Atlanta airport the emp...       1
...                                                  ...     ...
14995               Very fast air line services in world       5
14996  It’s a really nice airport, new, clean and eff...       5
14997        Pleased with all the different food options       5
14998  (Translated by Google) Is very clear (Original...       5
14999         One Big Airport.  Got lost was super clean       5

[15000 rows x 2 columns]


In [ ]:
df = big_df.copy()

In [ ]:
df.head()

,review_comment,rating
0,Lost in Hartsfield I swear I hate ATL airport ...,1
1,Atlanta airport trash 😫,1
2,"flight got fucking canceled, might have to spe...",1
3,"Disgusting. Took one bite, spit it out and th...",1
4,don’t ever get lost in Atlanta airport the emp...,1


In [ ]:
# 3 star ratings
# Define a dictionary that maps the original ratings to the new ratings
rating_map = {
    1: -1,
    2: -1,
    3: 0,
    4: 1,
    5: 1
}

In [ ]:
# Create a new column by applying the map function to the rating column
df["new_rating"] = df["rating"].map(rating_map)
# Convert rating to categorical variable
df["new_rating"] = df["new_rating"].astype("category")
# Convert rating to categorical variable
df["rating"] = df["rating"].astype("category")
# convert review_comment column to text
df["review_comment"] = df["review_comment"].astype(str)

In [ ]:
df["review_comment"] = df["review_comment"].apply(str)
df["review_comment"] = df["review_comment"].astype("U")
print(df["review_comment"].dtype)

object


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   review_comment  15000 non-null  object  
 1   rating          15000 non-null  category
 2   new_rating      15000 non-null  category
dtypes: category(2), object(1)
memory usage: 146.9+ KB


In [ ]:
# Split data into train and test sets
train_df = df.sample(frac=0.8, random_state=42) # use 80% of data for training
test_df = df.drop(train_df.index) # use the rest for testing

In [ ]:
# Create unigram features using CountVectorizer - 3 star
vectorizer = CountVectorizer(ngram_range=(1,1))
X_train = vectorizer.fit_transform(train_df["review_comment"])
X_test = vectorizer.transform(test_df["review_comment"])
y_train = train_df["new_rating"]
y_test = test_df["new_rating"]

In [ ]:
# Fit a multinomial logistic regression model
model = LogisticRegression(multi_class="multinomial", solver="lbfgs",max_iter = 1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [ ]:
# Make predictions on test set
y_pred = model.predict(X_test)

# Evaluate accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 73.13%


In [ ]:
from sklearn.metrics import classification_report


In [ ]:
target_names = ['-1','0','1']
report = classification_report(y_test, y_pred, target_names=target_names)
print(report)

              precision    recall  f1-score   support

          -1       0.75      0.83      0.79      1161
           0       0.48      0.35      0.41       607
           1       0.80      0.82      0.81      1232

    accuracy                           0.73      3000
   macro avg       0.68      0.67      0.67      3000
weighted avg       0.72      0.73      0.72      3000



In [ ]:
# Create unigram features using CountVectorizer - 5 star
vectorizer = CountVectorizer(ngram_range=(1,1))
X_train = vectorizer.fit_transform(train_df["review_comment"])
X_test = vectorizer.transform(test_df["review_comment"])
y_train = train_df["rating"]
y_test = test_df["rating"]


In [ ]:
# Fit a multinomial logistic regression model
model = LogisticRegression(multi_class="multinomial", solver="lbfgs",max_iter = 1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [ ]:
# Make predictions on test set
y_pred = model.predict(X_test)

# Evaluate accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 56.67%


In [ ]:
from sklearn.metrics import classification_report
target_names = ['1','2','3','4','5']
report = classification_report(y_test, y_pred, target_names=target_names)
print(report)

              precision    recall  f1-score   support

           1       0.61      0.61      0.61       577
           2       0.45      0.49      0.47       584
           3       0.47      0.44      0.45       607
           4       0.54      0.51      0.53       610
           5       0.75      0.78      0.76       622

    accuracy                           0.57      3000
   macro avg       0.56      0.57      0.56      3000
weighted avg       0.57      0.57      0.57      3000



In [ ]:
# Bigram
# Create unigram features using CountVectorizer - 3 star
vectorizer = CountVectorizer(ngram_range=(2,2))
X_train = vectorizer.fit_transform(train_df["review_comment"])
X_test = vectorizer.transform(test_df["review_comment"])
y_train = train_df["new_rating"]
y_test = test_df["new_rating"]

In [ ]:
# Fit a multinomial logistic regression model
model = LogisticRegression(multi_class="multinomial", solver="lbfgs",max_iter = 1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [ ]:
# Make predictions on test set
y_pred = model.predict(X_test)

# Evaluate accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 69.43%


In [ ]:
target_names = ['-1','0','1']
report = classification_report(y_test, y_pred, target_names=target_names)
print(report)

              precision    recall  f1-score   support

          -1       0.67      0.84      0.74      1161
           0       0.51      0.24      0.33       607
           1       0.76      0.78      0.77      1232

    accuracy                           0.69      3000
   macro avg       0.65      0.62      0.61      3000
weighted avg       0.68      0.69      0.67      3000



In [ ]:
# Create bigram features using CountVectorizer - 5 star
vectorizer = CountVectorizer(ngram_range=(2,2))
X_train = vectorizer.fit_transform(train_df["review_comment"])
X_test = vectorizer.transform(test_df["review_comment"])
y_train = train_df["rating"]
y_test = test_df["rating"]


In [ ]:
# Fit a multinomial logistic regression model
model = LogisticRegression(multi_class="multinomial", solver="lbfgs",max_iter = 1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [ ]:
# Make predictions on test set
y_pred = model.predict(X_test)

# Evaluate accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 50.50%


In [ ]:
from sklearn.metrics import classification_report
target_names = ['1','2','3','4','5']
report = classification_report(y_test, y_pred, target_names=target_names)
print(report)

              precision    recall  f1-score   support

           1       0.51      0.54      0.52       577
           2       0.38      0.44      0.41       584
           3       0.45      0.38      0.41       607
           4       0.52      0.45      0.48       610
           5       0.67      0.70      0.68       622

    accuracy                           0.51      3000
   macro avg       0.50      0.50      0.50      3000
weighted avg       0.51      0.51      0.50      3000



In [ ]:
# Create Trigram features using CountVectorizer - 3 star
vectorizer = CountVectorizer(ngram_range=(3,3))
X_train = vectorizer.fit_transform(train_df["review_comment"])
X_test = vectorizer.transform(test_df["review_comment"])
y_train = train_df["new_rating"]
y_test = test_df["new_rating"]

In [ ]:
# Fit a multinomial logistic regression model
model = LogisticRegression(multi_class="multinomial", solver="lbfgs",max_iter = 1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [ ]:
# Make predictions on test set
y_pred = model.predict(X_test)

# Evaluate accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))


Accuracy: 60.53%


In [ ]:
target_names = ['-1','0','1']
report = classification_report(y_test, y_pred, target_names=target_names)
print(report)

              precision    recall  f1-score   support

          -1       0.54      0.91      0.68      1161
           0       0.56      0.08      0.14       607
           1       0.75      0.58      0.65      1232

    accuracy                           0.61      3000
   macro avg       0.62      0.52      0.49      3000
weighted avg       0.63      0.61      0.56      3000



In [ ]:
# Create Trigram features using CountVectorizer - 5 star
vectorizer = CountVectorizer(ngram_range=(3,3))
X_train = vectorizer.fit_transform(train_df["review_comment"])
X_test = vectorizer.transform(test_df["review_comment"])
y_train = train_df["rating"]
y_test = test_df["rating"]

In [ ]:
# Fit a multinomial logistic regression model
model = LogisticRegression(multi_class="multinomial", solver="lbfgs",max_iter = 1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [ ]:
# Make predictions on test set
y_pred = model.predict(X_test)

# Evaluate accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 40.30%


In [ ]:
from sklearn.metrics import classification_report
target_names = ['1','2','3','4','5']
report = classification_report(y_test, y_pred, target_names=target_names)
print(report)

              precision    recall  f1-score   support

           1       0.31      0.68      0.42       577
           2       0.36      0.26      0.30       584
           3       0.41      0.25      0.31       607
           4       0.48      0.32      0.39       610
           5       0.60      0.51      0.55       622

    accuracy                           0.40      3000
   macro avg       0.43      0.40      0.39      3000
weighted avg       0.44      0.40      0.40      3000



In [ ]:
# Create four-gram features using CountVectorizer - 3 star
vectorizer = CountVectorizer(ngram_range=(4,4))
X_train = vectorizer.fit_transform(train_df["review_comment"])
X_test = vectorizer.transform(test_df["review_comment"])
y_train = train_df["new_rating"]
y_test = test_df["new_rating"]

In [ ]:
# Fit a multinomial logistic regression model
model = LogisticRegression(multi_class="multinomial", solver="lbfgs",max_iter = 1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [ ]:
# Make predictions on test set
y_pred = model.predict(X_test)

# Evaluate accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))


Accuracy: 51.37%


In [ ]:
target_names = ['-1','0','1']
report = classification_report(y_test, y_pred, target_names=target_names)
print(report)

              precision    recall  f1-score   support

          -1       0.45      0.96      0.62      1161
           0       0.56      0.02      0.05       607
           1       0.79      0.33      0.47      1232

    accuracy                           0.51      3000
   macro avg       0.60      0.44      0.38      3000
weighted avg       0.61      0.51      0.44      3000



In [ ]:
# Create Trigram features using CountVectorizer - 5 star
vectorizer = CountVectorizer(ngram_range=(4,4))
X_train = vectorizer.fit_transform(train_df["review_comment"])
X_test = vectorizer.transform(test_df["review_comment"])
y_train = train_df["rating"]
y_test = test_df["rating"]

In [ ]:
# Fit a multinomial logistic regression model
model = LogisticRegression(multi_class="multinomial", solver="lbfgs",max_iter = 1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [ ]:
# Make predictions on test set
y_pred = model.predict(X_test)

# Evaluate accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 30.33%


In [ ]:
from sklearn.metrics import classification_report
target_names = ['1','2','3','4','5']
report = classification_report(y_test, y_pred, target_names=target_names)
print(report)

              precision    recall  f1-score   support

           1       0.23      0.88      0.37       577
           2       0.33      0.09      0.14       584
           3       0.39      0.11      0.17       607
           4       0.55      0.20      0.29       610
           5       0.59      0.26      0.36       622

    accuracy                           0.30      3000
   macro avg       0.42      0.31      0.27      3000
weighted avg       0.42      0.30      0.27      3000



# Pre-processing


In [ ]:
# Remove weblinks

def remove_hyperlinks(text):
  text = str(text)
  # use a regular expression to find and replace all hyperlinks with an empty string
  # the pattern matches any string that starts with http:// or https:// followed by any characters until a whitespace or end of string
  # the flags argument makes the pattern case-insensitive
  return re.sub(r"http[s]?://\S+", "", text, flags=re.I)


In [ ]:
# define a function to remove the strings and everything after them
def remove_translated_and_original_and_after(text):
  # check if the text contains both strings
  if "(Translated by Google)" in text and "(Original)" in text:
    # split the text by the '(original)' keyword and take the first part
    # remove any leading or trailing whitespace
    return text.split("(Original)")[0].strip()
  else:
    # return the text as it is
    return text

In [ ]:
# Handling Emojis

# define a function to replace emojis with their names
def replace_emojis(text):
  # use the demojize function to convert emojis to text
  # use the delimiters parameter to add spaces around the text
  return emoji.demojize(text, delimiters=(" ", " "))

In [ ]:
# define a function to remove special characters
def remove_special_characters(text):
  # use the sub function to replace any non-alphanumeric character with an empty string
  # use the flags parameter to ignore case
  return re.sub(r"[^a-zA-Z0-9]", "", text, flags=re.IGNORECASE)

In [ ]:
df.head()

,review_comment,rating,new_rating
0,Lost in Hartsfield I swear I hate ATL airport ...,1,-1
1,Atlanta airport trash 😫,1,-1
2,"flight got fucking canceled, might have to spe...",1,-1
3,"Disgusting. Took one bite, spit it out and th...",1,-1
4,don’t ever get lost in Atlanta airport the emp...,1,-1


In [ ]:
df['processed_comment'] = df['review_comment'].apply(remove_hyperlinks)

In [ ]:
# apply the function to the review_comments column and assign it back
df["processed_comment"] = df["processed_comment"].apply(remove_translated_and_original_and_after)

In [ ]:
# apply the function to the review_comments column and assign it back
df["processed_comment"] = df["processed_comment"].apply(replace_emojis)

In [ ]:
#df["processed_comment"] = df["processed_comment"].apply(remove_special_characters)

In [ ]:
df.head(10)

,review_comment,rating,new_rating,weblink_rmvd_cmnt,processed_comment
0,Lost in Hartsfield I swear I hate ATL airport ...,1,-1,Lost in Hartsfield I swear I hate ATL airport ...,LostinHartsfieldIswearIhateATLairportwitapassion
1,Atlanta airport trash tired_face,1,-1,Atlanta airport trash 😫,Atlantaairporttrashtiredface
2,"flight got fucking canceled, might have to spe...",1,-1,"flight got fucking canceled, might have to spe...",flightgotfuckingcanceledmighthavetospend17hour...
3,"Disgusting. Took one bite, spit it out and th...",1,-1,"Disgusting. Took one bite, spit it out and th...",DisgustingTookonebitespititoutandthrewawayItta...
4,don’t ever get lost in Atlanta airport the emp...,1,-1,don’t ever get lost in Atlanta airport the emp...,dontevergetlostinAtlantaairporttheemployeeswil...
5,"Poor service, too much trolley around, queue 2...",1,-1,"Poor service, too much trolley around, queue 2...",Poorservicetoomuchtrolleyaroundqueue25mintoget...
6,security is incompetent and rude.,1,-1,security is incompetent and rude.,securityisincompetentandrude
7,"Do not waste your time or money , horrible ser...",1,-1,"Do not waste your time or money , horrible ser...",Donotwasteyourtimeormoneyhorribleservice
8,IMPOSSIBLE TO GET IN CONTACT WITH SOMEONE ON T...,1,-1,IMPOSSIBLE TO GET IN CONTACT WITH SOMEONE ON T...,IMPOSSIBLETOGETINCONTACTWITHSOMEONEONTHEWEEKEND
9,Everything is huge and nothing works.,1,-1,Everything is huge and nothing works.,Everythingishugeandnothingworks


In [ ]:
print(df)

                                          review_comment rating new_rating  \
0      Lost in Hartsfield I swear I hate ATL airport ...      1         -1   
1                     Atlanta airport trash  tired_face       1         -1   
2      flight got fucking canceled, might have to spe...      1         -1   
3      Disgusting.  Took one bite, spit it out and th...      1         -1   
4      don’t ever get lost in Atlanta airport the emp...      1         -1   
...                                                  ...    ...        ...   
14995               Very fast air line services in world      5          1   
14996  It’s a really nice airport, new, clean and eff...      5          1   
14997        Pleased with all the different food options      5          1   
14998                       Is very clear  Is very clear      5          1   
14999         One Big Airport.  Got lost was super clean      5          1   

                                       weblink_rmvd_cmnt  \
0  

In [ ]:
# create a pipeline with three steps: 
# 1. apply the remove_hyperlinks function to the review_comment column using FunctionTransformer
# 2. transform the text into a matrix of unigram counts using CountVectorizer
# 3. fit and predict a logistic regression model using LogisticRegression
pipe = Pipeline([
  ("remove_hyperlinks", FunctionTransformer(remove_hyperlinks)),
  ("vectorizer", CountVectorizer(ngram_range=(1,1))),
  ("classifier", LogisticRegression(max_iter=1000))
])

In [ ]:
print(train_df["review_comment"].dtype)
print(train_df["review_comment"].isnull().any())
print(train_df["review_comment"].isna().any())

object
False
False


In [ ]:
# fit the pipeline on the training data
pipe.fit(train_df['review_comment'], train_df['rating'])

ValueError: ignored

In [ ]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Define some sample text documents
text = ["This is a sample sentence.",
        "Another sample sentence with different words.",
        "This is the third sample sentence."]

# Create an instance of CountVectorizer with ngram_range=(1,1) for unigrams
vectorizer = CountVectorizer(ngram_range=(1,1))

# Fit and transform the text documents into a matrix of token counts
matrix = vectorizer.fit_transform(text)

# Convert the matrix to an array
array = matrix.toarray()

# Get the feature names (unigrams)
features = vectorizer.get_feature_names_out()

# Create a dataframe with the array and feature names as columns
import pandas as pd
df = pd.DataFrame(data=array, columns=features)

# Display the dataframe
print(df)

   another  different  is  sample  sentence  the  third  this  with  words
0        0          0   1       1         1    0      0     1     0      0
1        1          1   0       1         1    0      0     0     1      1
2        0          0   1       1         1    1      1     1     0      0


In [ ]:
%%time
# Create an instance of GridSearchCV with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5)

# Fit the grid search on the data
grid_search.fit(X_train, y_train)

ValueError: ignored

In [ ]:
# Define a parameter grid to search over
param_grid = {
    "vect__ngram_range": [(1,1)], # use unigrams
    "clf__C": [0.001, 0.01, 0.1, 1, 10, 100, 1000], # inverse regularization strength
    "clf__penalty": ["l1", "l2"] # type of regularization
}

In [ ]:
# Print the best score and parameters
print("Best score: {:.2f}".format(grid_search.best_score_))
print("Best parameters: {}".format(grid_search.best_params_))

AttributeError: ignored